In [2]:
from sklearn.externals import joblib

In [3]:
## load test model
clf = joblib.load('classify_test.model') # 분류기 엔진
cate_dict = joblib.load('cate_dict.dat') # id별 카테고리가 맵핑되어있는 파일
vectorizer = joblib.load('vectorizer_test.dat') # 제목들을 숫자 id로 변환해주는 것

In [4]:
cate_id_name_dict = dict(map(lambda (k,v):(v,k),cate_dict.items()))

In [5]:
# initialize size
size = 0
joblib.dump(size,'size.dat')

['size.dat']

In [6]:
# read json results of samples
list_caffe_samples = joblib.load('list_caffe_samples.dat')
list_caffe_test = joblib.load('list_caffe_test.dat')

is_test = False

if is_test :
    list_caffe = list_caffe_test
else :
    list_caffe = list_caffe_samples
print len(list_caffe)

206


In [7]:
# define removeStopWords

mapping = { 'A/S':'AS'}
stop_words = [u"%즉시할인쿠폰",  u"5% 즉시할인", u"0% 즉시할인", u"% 즉시할인", u"5% 할인", u"0% 할인", u"5%할인", u"0%할인", u"% 할인", u"적립",
              u"쿠폰", u"즉시",u"정품", u"증정", u"할인",
              u"해외직배송", u"직배송", u"무료배송", u"안전배송", u"당일", u"배송", u"발송", u"빠른",
              u"신한카드", u"BC카드", u"KB국민카드", u"국민카드", u"롯데카드", u"KB카드", u"현대카드", u"삼성카드", u"일까지"]
symbols = ["[","]","(",")", "+", "-", "/", "?", "~", "*", u"☆", u"★", u"●", u"♥", u"■", u"▶", u"◀", u"↑"]
ambg_words = [u"해외", u"구매대행"]

start = 0
length = 1
remove_ambg = True

# define concatenate function
def concate(list1, list2) :
    for each in list2:
        list1.append(each)
    return list1

remove_list = concate(stop_words, symbols)
if remove_ambg:
    remove_list = concate(remove_list,ambg_words)
    
def removeStopWords(word) :
     # replace
    for before, after in mapping.iteritems():
        word = word.replace(before, after)
    #remove
    for each in remove_list :
        word = word.replace(each," ")
    return word

In [8]:
# define splitMorph using konlpy

from konlpy.tag import Twitter
from konlpy.utils import pprint
kkma = Twitter()

def splitMorph(word) :
    morphs = kkma.pos(word)
    words = ""
    for morph in morphs :
        word = morph[0]
        words += " " + word
    return words

In [ ]:
# server for 206 samples tests

from bottle import route, run, template,request,get, post
import  time
from threading import  Condition
_CONDITION = Condition()

# initialize size
size = 0
joblib.dump(size,'size.dat')

@route('/classify')
def classify():
    print "classify called"
    img = request.GET.get('img','')
    name = request.GET.get('name', '').decode('utf-8')
    
    name = removeStopWords(name)
    name = splitMorph(name)
    
    # read size
    size = joblib.load('size.dat')
    
    # append caffe result
    res = list_caffe_samples[size]['result']
    if res[0] :
        for each in res[1] :
            name += " " + each[0].replace(" ", "")
        for each in res[2] :
            name += " " + each[0].replace(" ", "")
    
    vec = vectorizer.transform([name])
    p = clf.predict(vec)
    pred = p[0]
    
    # update and store size
    if size % 100 == 0:
        print "%d saved" % size
        print name
        print cate_id_name_dict[pred]
    
    size = size + 1;
    joblib.dump(size,'size.dat')
    
    return {'cate':cate_id_name_dict[pred]}


run(host='0.0.0.0', port=8887)
#  http://somaeval.hoot.co.kr:8884/eval?url=http://  :8887&mode=all&name=oyj

Bottle v0.12.9 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8887/
Hit Ctrl-C to quit.



classify called
0 saved
 ' 안녕 안녕 안녕 안녕 ' website BandAid packet cashmachine envelope website computer machine device container
디지털/가전;PC부품;CPU


192.168.99.1 - - [22/Aug/2016 15:35:17] "GET /classify?name=%27%EC%95%88%EB%85%95%EC%95%88%EB%85%95%20%EC%95%88%EB%85%95%20%EC%95%88%EB%85%95%27&img=%27asdasdadsad%27 HTTP/1.1" 200 62
192.168.99.1 - - [22/Aug/2016 15:35:17] "GET /favicon.ico HTTP/1.1" 404 747


 * 추후 여기 docker 에서 뭔가 python package 설치할게 있으면 
 * /opt/conda/bin/pip2 install bottle 이런식으로 설치 가능